# Create training data with training variables function
#### To be merged with current year good_df and then passed into relevent machine learning functions for training. The bad_df will be used as the test_df. 

In [ ]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
import pyarrow as pa
import pyarrow.parquet as pq
import gcsfs
import getpass
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import geopandas as gpd
import sgis as sg
import dapla as dp
import datetime
from dapla.auth import AuthClient
from dapla import FileClient

fs = FileClient.get_gcs_file_system()
import numpy as np


import warnings

warnings.filterwarnings("ignore")

In [ ]:
# import datetime

# # Get the current date
# current_date = datetime.datetime.now()

# # Format the year and month
# current_year = current_date.strftime("%Y")
# current_month = current_date.strftime("%m")

# # Subtract one day from the first day of the current month to get the last day of the previous month
# last_day_of_previous_month = datetime.datetime(
#     current_date.year, current_date.month, 1
# ) - datetime.timedelta(days=1)

# # Now we can get the month number of the previous month
# previous_month = last_day_of_previous_month.strftime("%m")

# VOFSTI = (
#     "ssb-vof-data-delt-prod/stedfesting-situasjonsuttak_data/klargjorte-data/parquet"
# )
# file_path = (
#     f"{VOFSTI}/stedfesting-situasjonsuttak_p{current_year}-{previous_month}_v1.parquet"
# )

# vof_df = dp.read_pandas(f"{file_path}")
# vof_gdf = gpd.GeoDataFrame(
#     vof_df,
#     geometry=gpd.points_from_xy(
#         vof_df["y_koordinat"],
#         vof_df["x_koordinat"],
#     ),
#     crs=25833,
# )

# vof_gdf = vof_gdf.rename(
#     columns={
#         "orgnrbed": "orgnr_bedrift",
#         "org_nr": "orgnr_foretak",
#         "nace1_sn07": "naring",
#     }
# )


# vof_gdf = vof_gdf[
#     [
#         "orgnr_bedrift",
#         "orgnr_foretak",
#         "naring",
#         "x_koordinat",
#         "y_koordinat",
#         "rute_100m",
#         "rute_1000m",
#         "geometry",
#     ]
# ]
# pd.set_option("display.max_columns", None)

# vof_gdf = vof_gdf.dropna(subset=["x_koordinat"])
# vof_gdf = vof_gdf.drop_duplicates(subset="orgnr_bedrift")
# vof_gdf = vof_gdf.drop("orgnr_foretak", axis=1)
# vof_gdf.head()

In [ ]:
import pandas as pd

# Define a list of years
years_to_process = [2020, 2021]

# Define the values for naring_f_3 to filter on
varehandel = ["45", "46", "47"]

# Initialize an empty list to store dataframes for each iteration
dfs = []

for year in years_to_process:
    fil_path = f"gs://ssb-prod-noeku-data-produkt/statistikkfiler/g{year}/statistikkfil_bedrifter_pub.parquet"

    # Read data for the current year
    bedrifter = pd.read_parquet(fil_path, filesystem=fs)

    # Create 'nace4' by slicing the first 5 characters of 'naring'
    bedrifter["naring4"] = bedrifter["naring"].str[:5]

    bedrifter["naring_f_4"] = bedrifter["naring_f"].str[:5]

    # Create 'nace3' by slicing the first 4 characters of 'naring'
    bedrifter["naring3"] = bedrifter["naring"].str[:4]

    bedrifter["naring_f_3"] = bedrifter["naring_f"].str[:4]

    bedrifter["naring_f_2"] = bedrifter["naring_f"].str[:2]

    # Filter rows based on desired naring_f_3 values
    filtered_bedrifter = bedrifter[bedrifter["naring_f_2"].isin(varehandel)]

    # Append the resulting dataframe to the list
    dfs.append(filtered_bedrifter)

# Merge all dataframes in the list into a single dataframe
merged_dataframe = pd.concat(dfs, ignore_index=True)

# Display the resulting merged dataframe
print(merged_dataframe.head())

In [ ]:
import pandas as pd

# Define a list of years
years_to_process = [2020, 2021]

# Define the values for naring_f_3 to filter on
varehandel = [45, 46, 47]

# Initialize an empty list to store dataframes for each iteration
dfs = []

for year in years_to_process:
    fil_path = f"gs://ssb-prod-noeku-data-produkt/statistikkfiler/g{year}/statistikkfil_bedrifter_pub.parquet"

    # Read data for the current year
    bedrifter = pd.read_parquet(fil_path, filesystem=fs)

    # Create 'nace4' by slicing the first 5 characters of 'naring'
    bedrifter["naring4"] = bedrifter["naring"].str[:5]

    bedrifter["naring_f_4"] = bedrifter["naring_f"].str[:5]

    # Create 'nace3' by slicing the first 4 characters of 'naring'
    bedrifter["naring3"] = bedrifter["naring"].str[:4]

    bedrifter["naring_f_3"] = bedrifter["naring_f"].str[:4]

    # Filter rows based on desired naring_f_3 values
    filtered_bedrifter = bedrifter[
        bedrifter["naring_f_3"].isin(desired_naring_f_3_values)
    ]

    # Append the resulting dataframe to the list
    dfs.append(filtered_bedrifter)

# Merge all dataframes in the list into a single dataframe
merged_dataframe = pd.concat(dfs, ignore_index=True)

# Display the resulting merged dataframe
print(merged_dataframe.head())